In [9]:
best = open("human_hcd_tryp_best.msp")
c = 0
fields = ['modified_sequence', 'collision_energy', 'precursor_charge']
data = []
total = 0
names = []

while True:
    bestline = best.readline()
    
    if not bestline: break
    
    if bestline.startswith('Name: '):
        keep = True
        checkM = True
        total += 1
        #if not (c % 10): print("row ", c, " done")
        c += 1
        idx = [] # 추가
        name = bestline[6:-1] # AAAAAAAAEQQSSNGPVK/2_1(0,A,Acetyl)
        
        s = name.split('/')
        seq = s[0]            # AAAAAAAAEQQSSNGPVK
        charge = int(s[1][0])      # 2
        
        ### reverse seq
        re_seq = seq[:-1]
        re_seq = re_seq[::-1] + seq[-1]
        
        # modification 있다면?
        if name[-1] != '0':
            
            # ALSVQASLMTPVQPMLAEACK/2_3(8,M,Oxidation)(14,M,Oxidation)(19,C,CAM)
            #idx = []  # [8, 14]
            for i in range(0, len(name)):
                if not keep:
                    break
                if name[i] == '(':
                    mod = ''       # 8,M,Oxidation
                    
                    for j in range(i+1, len(name)):
                        if name[j] == ')': break
                        
                        mod += name[j]
                    
                    m = mod.split(',') # ['8', 'M', 'Oxidation']
                    
                    #print(m)
                    
                    # oxidation 이나 cam 아닌 다른 modification이면 날림
                    if m[2] != 'Oxidation' and m[2] != 'CAM':
                        #print(m[2])
                        keep = False
                        break
                    
                    if m[2] == 'Oxidation':
                        idx += [int(m[0])]
            
            if not keep:
                #print('이건 제거: ', re_seq)
                continue
            
            # idx = [8, 14] ----> idx = [len(seq)-2 - 8, len(seq)-2 - 14]
            for i in range(0, len(idx)):
                idx[i] = len(seq) - 2 - idx[i]
            idx.sort()
            
            for i in range(0, len(idx)):
                # 다음 인덱스들에 +4
                for j in range(i+1, len(idx)):
                    idx[j] += 4
                
                # 만약 마지막 아미노산에서 modification 일어났다면
                if idx[i] == -1:
                    # 만약 M(ox)가 맞다면
                    if re_seq[-1] == 'M':
                        re_seq = re_seq + '(ox)'
                    # 만약 M이 아니라면
                    else:
                        checkM = False
                
                # 중간 인덱스 자리에 '(ox)' 추가
                else:
                    # 만약 M(ox)가 맞다면
                    if re_seq[idx[i]] == 'M':
                        idx[i] += 1
                        re_seq = re_seq[:idx[i]] + '(ox)' + re_seq[idx[i]:]
                    # 만약 M이 아니라면
                    else:
                        checkM = False
                    
    # collision energy 찾기
    if bestline.startswith('Comment: ') and keep:
        pos_nce = bestline.find('NCE=')
        nce = round(float(bestline[pos_nce+4 : pos_nce+9]))
        
        # Prosit 조건.
        if len(re_seq) >= 7 or len(re_seq) <= 30:
            if nce >= 10 and charge <= 6:         # nce는 0이 있고, charge는 7,8이 있음
                if checkM:                         # M(ox)민 받고 나머지는 버림
                    data.append([re_seq, nce, charge])
                    #print(name)
                    names.append(name)
                    
best.close()
print(total)
print(names)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [6]:
names

['AAAAAAAAAAAAAK/2_0',
 'AAAAAAAAAAAAK/2_0',
 'AAAAAAAAAAAK/2_0',
 'AAAAAAAAAAGEGAR/2_0',
 'AAAAAAAAAAGR/2_0',
 'AAAAAAAAAGEGAR/2_0',
 'AAAAAAAAAGQIHHVTQNGGLYK/3_0',
 'AAAAAAAAAPAAAATAPTTA/2_0',
 'AAAAAAAAAVPR/2_0',
 'AAAAAAAAQAK/2_0',
 'AAAAAAAAVVR/2_0',
 'AAAAAAAAYSDGYGR/2_0',
 'AAAAAAALQAK/2_0',
 'AAAAAADGPPAADGEDGQDPHSK/3_0',
 'AAAAAADPNAAWAAYYSH/2_0',
 'AAAAAADPNAAWAAYYSH/3_0',
 'AAAAAADPNAAWAAYYSHY/2_0',
 'AAAAAADPNAAWAAYYSHYY/2_0',
 'AAAAAAGPAGAGGGR/2_0',
 'AAAAAALQAK/2_0',
 'AAAAAAPAPAPAPAPAPAPAKPK/3_0',
 'AAAAAAPAPAPAPAPAPAPAKPKEEK/3_0',
 'AAAAAAPAPAPAPAPAPAPAKPKEEK/4_0',
 'AAAAAAPPGLR/2_0',
 'AAAAAAQCRSPR/2_1(7,C,CAM)',
 'AAAAAAQQQQQCGGGGATKPAVSGK/3_1(11,C,CAM)',
 'AAAAAAQSVYAFSARPLAGGEPVSLGSLR/3_0',
 'AAAAAATPAVR/2_0',
 'AAAAAAVSGSAAAEAK/2_0',
 'AAAAAAVSGSAAAEAK/3_0',
 'AAAAAAYSDGYGR/2_0',
 'AAAAADGPPAADGEDGQDPHSK/2_0',
 'AAAAADGPPAADGEDGQDPHSK/3_0',
 'AAAAAEHSQR/2_0',
 'AAAAAGPAGAGGGR/2_0',
 'AAAAAGPGLGGSLGGALAGPLGGALGVPGAK/3_0',
 'AAAAAGQIHHVTQNGGLYK/3_0',
 'AAAAAGSGAGTGGG

In [8]:
with open('best.msp', 'w') as f:
    best = open("human_hcd_tryp_best.msp")
    
    index = 0
    
    #bestline = best.readline()
    while True:
        if index >= len(names): break
        
        bestline = best.readline()
    
        if not bestline: break
    
        if bestline.startswith('Name: '):
            name = bestline[6:-1]
            
            print(name)
            print('find: ', names[index])
            
            if name == names[index]:
                index += 1
                print(bestline[:-1])
                f.write(bestline)
                
                while True:
                    bestline = best.readline()
                    
                    if bestline.startswith('\n'): break
                    
                    
                    print(bestline[:-1])
                    f.write(bestline)


AAAAAAAAAAAAAAAGAGAGAK/2_0
find:  AAAAAAAAAAAAAK/2_0
AAAAAAAAAAAAAAAGAGAGAK/3_0
find:  AAAAAAAAAAAAAK/2_0
AAAAAAAAAAAPPAPPEGASPGDSAR/2_0
find:  AAAAAAAAAAAAAK/2_0
AAAAAAAAAAAPPAPPEGASPGDSAR/3_0
find:  AAAAAAAAAAAAAK/2_0
AAAAAAAAAASGAAIPPLIPPR/3_0
find:  AAAAAAAAAAAAAK/2_0
AAAAAAAAAVSR/2_0
find:  AAAAAAAAAAAAAK/2_0
AAAAAAAAEQQSSNGPVK/2_1(0,A,Acetyl)
find:  AAAAAAAAAAAAAK/2_0
AAAAAAAAGAFAGR/2_0
find:  AAAAAAAAAAAAAK/2_0
AAAAAAAAQMHAK/2_1(9,M,Oxidation)
find:  AAAAAAAAAAAAAK/2_0
AAAAAAAATMALAAPSSPTPESPTMLTK/3_0
find:  AAAAAAAAAAAAAK/2_0
AAAAAAAAVPSAGPAGPAPTSAAGR/2_0
find:  AAAAAAAAAAAAAK/2_0
AAAAAAAAVPSAGPAGPAPTSAAGR/3_0
find:  AAAAAAAAAAAAAK/2_0
AAAAAAAGAAGSAAPAAAAGAPGSGGAPSGSQGVLIGDR/4_1(0,A,Acetyl)
find:  AAAAAAAAAAAAAK/2_0
AAAAAAAGDSDSWDADAFSVEDPVR/2_1(0,A,Acetyl)
find:  AAAAAAAAAAAAAK/2_0
AAAAAAAGDSDSWDADAFSVEDPVR/3_1(0,A,Acetyl)
find:  AAAAAAAAAAAAAK/2_0
AAAAAAALESWQAAAPR/2_1(0,A,Acetyl)
find:  AAAAAAAAAAAAAK/2_0
AAAAAAASFAAEQAGAAALLPLGAAADHHSLYK/4_0
find:  AAAAAAAAAAAAAK/2_0
AAAAAA

AADPTFVMGHAMATGLVLIGTGSSVK/2_0
find:  AAAAAAAAAAAAAK/2_0
AADPTFVMGHAMATGLVLIGTGSSVK/3_0
find:  AAAAAAAAAAAAAK/2_0
AADPTFVMGHAMATGLVLIGTGSSVK/3_2(7,M,Oxidation)(11,M,Oxidation)
find:  AAAAAAAAAAAAAK/2_0
AADPVPSFDESSQDTSEK/2_0
find:  AAAAAAAAAAAAAK/2_0
AADPVSPLLHELTFQAMAYDLLDIEQDTYR/3_0
find:  AAAAAAAAAAAAAK/2_0
AADPVSPLLHELTFQAMAYDLLDIEQDTYR/3_1(16,M,Oxidation)
find:  AAAAAAAAAAAAAK/2_0
AADPWVLK/2_0
find:  AAAAAAAAAAAAAK/2_0
AADQFDIYSSQQSK/2_0
find:  AAAAAAAAAAAAAK/2_0
AADQRPK/2_1(0,A,Acetyl)
find:  AAAAAAAAAAAAAK/2_0
AADQTTAEQGMR/2_0
find:  AAAAAAAAAAAAAK/2_0
AADQTTAEQGMR/2_1(10,M,Oxidation)
find:  AAAAAAAAAAAAAK/2_0
AADRIHQDGIHILVNMNGYTK/4_0
find:  AAAAAAAAAAAAAK/2_0
AADRIHQDGIHILVNMNGYTK/4_1(15,M,Oxidation)
find:  AAAAAAAAAAAAAK/2_0
AADRLEEEIEEHAFDDNK/3_0
find:  AAAAAAAAAAAAAK/2_0
AADRLEQELLR/3_0
find:  AAAAAAAAAAAAAK/2_0
AADRLPNLSSPSAEGPPGPPSGPAPR/2_0
find:  AAAAAAAAAAAAAK/2_0
AADRLPNLSSPSAEGPPGPPSGPAPR/3_0
find:  AAAAAAAAAAAAAK/2_0
AADSDDGAVSAPAASDGGVSK/2_0
find:  AAAAAAAAAAAAAK/2_

find:  AAAAAAAAAAAAAK/2_0
AAGSSGFEAVEVAWPYAETPEALAR/2_0
find:  AAAAAAAAAAAAAK/2_0
AAGSSGFEAVEVAWPYAETPEALAR/3_0
find:  AAAAAAAAAAAAAK/2_0
AAGSTVR/2_0
find:  AAAAAAAAAAAAAK/2_0
AAGSYGGMASPMGVYSGHPEQYSAGMGR/3_0
find:  AAAAAAAAAAAAAK/2_0
AAGTAAALAFLSQESR/2_1(0,A,Acetyl)
find:  AAAAAAAAAAAAAK/2_0
AAGTAGQLLYSNLQHLK/2_0
find:  AAAAAAAAAAAAAK/2_0
AAGTAGQLLYSNLQHLK/3_0
find:  AAAAAAAAAAAAAK/2_0
AAGTALGQALR/2_0
find:  AAAAAAAAAAAAAK/2_0
AAGTCGVLLR/2_1(4,C,CAM)
find:  AAAAAAAAAAAAAK/2_0
AAGTDGNALQDQQLSK/2_0
find:  AAAAAAAAAAAAAK/2_0
AAGTDGSDFQHR/2_0
find:  AAAAAAAAAAAAAK/2_0
AAGTDGSDFQHR/3_0
find:  AAAAAAAAAAAAAK/2_0
AAGTDGSDFQHRER/3_0
find:  AAAAAAAAAAAAAK/2_0
AAGTDSFNGHPPQGCASTPVAR/3_1(14,C,CAM)
find:  AAAAAAAAAAAAAK/2_0
AAGTEGPFQEVDGVATTR/2_0
find:  AAAAAAAAAAAAAK/2_0
AAGTGFTVNVAWNGPR/2_0
find:  AAAAAAAAAAAAAK/2_0
AAGTLYTYPENWR/2_0
find:  AAAAAAAAAAAAAK/2_0
AAGTLYTYPENWR/2_1(0,A,Acetyl)
find:  AAAAAAAAAAAAAK/2_0
AAGTLYTYPENWR/3_1(0,A,Acetyl)
find:  AAAAAAAAAAAAAK/2_0
AAGTSGLILK/2_0
find:  AAA

AAMALEK/2_0
find:  AAAAAAAAAAAAAK/2_0
AAMAPALTDAAAEAHHIR/3_1(0,A,Acetyl)
find:  AAAAAAAAAAAAAK/2_0
AAMAQVIGDRVPK/3_0
find:  AAAAAAAAAAAAAK/2_0
AAMAQVIGDRVPK/3_1(2,M,Oxidation)
find:  AAAAAAAAAAAAAK/2_0
AAMAQWR/2_0
find:  AAAAAAAAAAAAAK/2_0
AAMAVGGAGGSR/2_1(0,A,Acetyl)
find:  AAAAAAAAAAAAAK/2_0
AAMAVGGAGGSR/2_2(0,A,Acetyl)(2,M,Oxidation)
find:  AAAAAAAAAAAAAK/2_0
AAMDAVCAK/2_1(6,C,CAM)
find:  AAAAAAAAAAAAAK/2_0
AAMDFPGAK/2_0
find:  AAAAAAAAAAAAAK/2_0
AAMDFPGAK/2_1(2,M,Oxidation)
find:  AAAAAAAAAAAAAK/2_0
AAMDMTVFLK/2_0
find:  AAAAAAAAAAAAAK/2_0
AAMDNSEIAGEK/2_0
find:  AAAAAAAAAAAAAK/2_0
AAMDNSEIAGEK/2_1(2,M,Oxidation)
find:  AAAAAAAAAAAAAK/2_0
AAMDSAR/2_0
find:  AAAAAAAAAAAAAK/2_0
AAMEALVVEVTK/2_0
find:  AAAAAAAAAAAAAK/2_0
AAMEALVVEVTK/2_1(2,M,Oxidation)
find:  AAAAAAAAAAAAAK/2_0
AAMEALVVEVTK/3_0
find:  AAAAAAAAAAAAAK/2_0
AAMEFHEHLHSIGTK/4_0
find:  AAAAAAAAAAAAAK/2_0
AAMEMVPVLK/2_0
find:  AAAAAAAAAAAAAK/2_0
AAMEMVPVLK/2_1(2,M,Oxidation)
find:  AAAAAAAAAAAAAK/2_0
AAMEMVPVLK/2_2(2,M,Oxida

AASAGQEPLHNEELAGAGR/3_0
find:  AAAAAAAAAAAAAK/2_0
AASAGVPYHGEVPVSLAR/2_0
find:  AAAAAAAAAAAAAK/2_0
AASAGVPYHGEVPVSLAR/3_0
find:  AAAAAAAAAAAAAK/2_0
AASAHAIGTVK/2_0
find:  AAAAAAAAAAAAAK/2_0
AASAIYSGYIFDYDYYR/2_0
find:  AAAAAAAAAAAAAK/2_0
AASAIYSGYIFDYDYYRDDFYDR/3_0
find:  AAAAAAAAAAAAAK/2_0
AASALYACTK/2_2(0,A,Acetyl)(7,C,CAM)
find:  AAAAAAAAAAAAAK/2_0
AASAMDAVSR/2_1(4,M,Oxidation)
find:  AAAAAAAAAAAAAK/2_0
AASAPPPPDKLEGGGGPAPPPAPPSTGR/2_1(0,A,Acetyl)
find:  AAAAAAAAAAAAAK/2_0
AASAPYFEVLEK/2_0
find:  AAAAAAAAAAAAAK/2_0
AASASESALQTVIK/2_0
find:  AAAAAAAAAAAAAK/2_0
AASASESALQTVIKEDLPR/2_0
find:  AAAAAAAAAAAAAK/2_0
AASASESALQTVIKEDLPR/3_0
find:  AAAAAAAAAAAAAK/2_0
AASASSVHVPVSALGAGSAATAASEEMYVPR/3_0
find:  AAAAAAAAAAAAAK/2_0
AASAVAVCSFLCFCR/2_3(7,C,CAM)(11,C,CAM)(13,C,CAM)
find:  AAAAAAAAAAAAAK/2_0
AASAVGATAVETSR/2_0
find:  AAAAAAAAAAAAAK/2_0
AASAYAVGDVK/2_0
find:  AAAAAAAAAAAAAK/2_0
AASAYDYGSSHGLTDSSLLLDDYSSK/3_0
find:  AAAAAAAAAAAAAK/2_0
AASCVLLHTGQK/2_2(0,A,Acetyl)(3,C,CAM)
find:  AAAAA

AAVWEITGAEFSK/2_0
find:  AAAAAAAAAAAAAK/2_0
AAVWEVDFPDVAR/2_0
find:  AAAAAAAAAAAAAK/2_0
AAVWQQVLAVDAR/2_1(0,A,Acetyl)
find:  AAAAAAAAAAAAAK/2_0
AAVWTDSR/2_0
find:  AAAAAAAAAAAAAK/2_0
AAVYHHFISDGVR/2_0
find:  AAAAAAAAAAAAAK/2_0
AAVYHHFISDGVR/3_0
find:  AAAAAAAAAAAAAK/2_0
AAVYHHFISDGVR/4_0
find:  AAAAAAAAAAAAAK/2_0
AAVYLLDDPLAALDAHVGQHVFNQVIGPGGLLQGTTR/4_0
find:  AAAAAAAAAAAAAK/2_0
AAVYQPQPHPQPPPYGHCVTDSGVVYSVGMQWLK/3_1(17,C,CAM)
find:  AAAAAAAAAAAAAK/2_0
AAVYQPQPHPQPPPYGHCVTDSGVVYSVGMQWLK/3_2(17,C,CAM)(29,M,Oxidation)
find:  AAAAAAAAAAAAAK/2_0
AAVYQPQPHPQPPPYGHCVTDSGVVYSVGMQWLK/4_1(17,C,CAM)
find:  AAAAAAAAAAAAAK/2_0
AAVYQPQPHPQPPPYGHCVTDSGVVYSVGMQWLK/4_2(17,C,CAM)(29,M,Oxidation)
find:  AAAAAAAAAAAAAK/2_0
AAVYSGISLK/2_1(0,A,Acetyl)
find:  AAAAAAAAAAAAAK/2_0
AAVYVLIR/2_0
find:  AAAAAAAAAAAAAK/2_0
AAWAEYEAEK/2_0
find:  AAAAAAAAAAAAAK/2_0
AAWAQGYTGHGIVVSILDDGIEK/2_0
find:  AAAAAAAAAAAAAK/2_0
AAWAQGYTGHGIVVSILDDGIEK/3_0
find:  AAAAAAAAAAAAAK/2_0
AAWDIVSNAEK/2_0
find:  AAAAAAAAAAAAAK/2_0
AA

ACTSGDLPGCSCGPVPGEPPGPGNR/2_3(1,C,CAM)(9,C,CAM)(11,C,CAM)
find:  AAAAAAAAAAAAAK/2_0
ACTSQGCGKPITEESSTLGEGSK/2_2(1,C,CAM)(6,C,CAM)
find:  AAAAAAAAAAAAAK/2_0
ACTSQGCGKPITEESSTLGEGSK/3_2(1,C,CAM)(6,C,CAM)
find:  AAAAAAAAAAAAAK/2_0
ACTTEEDQEK/2_1(1,C,CAM)
find:  AAAAAAAAAAAAAK/2_0
ACTYIR/2_1(1,C,CAM)
find:  AAAAAAAAAAAAAK/2_0
ACVCQTLGISPEEK/2_2(1,C,CAM)(3,C,CAM)
find:  AAAAAAAAAAAAAK/2_0
ACVFGNEPK/2_1(1,C,CAM)
find:  AAAAAAAAAAAAAK/2_0
ACVGDVQER/2_1(1,C,CAM)
find:  AAAAAAAAAAAAAK/2_0
ACVGLPPQNHMLLEHK/2_1(1,C,CAM)
find:  AAAAAAAAAAAAAK/2_0
ACVGLPPQNHMLLEHK/3_1(1,C,CAM)
find:  AAAAAAAAAAAAAK/2_0
ACVGLPPQNHMLLEHK/3_2(1,C,CAM)(10,M,Oxidation)
find:  AAAAAAAAAAAAAK/2_0
ACVGLPPQNHMLLEHK/4_1(1,C,CAM)
find:  AAAAAAAAAAAAAK/2_0
ACVGLPPQNHMLLEHK/4_2(1,C,CAM)(10,M,Oxidation)
find:  AAAAAAAAAAAAAK/2_0
ACVGNAYHK/2_1(1,C,CAM)
find:  AAAAAAAAAAAAAK/2_0
ACVIAMR/2_1(1,C,CAM)
find:  AAAAAAAAAAAAAK/2_0
ACVIHGTDLK/2_1(1,C,CAM)
find:  AAAAAAAAAAAAAK/2_0
ACVIHGTDLKDFTSEQIDEILQNHTEIVFAR/4_1(1,C,CAM)
find:  AAAAA

ADIESLVTEK/2_0
find:  AAAAAAAAAAAAAK/2_0
ADIEVACYGYEGIDAVK/2_1(6,C,CAM)
find:  AAAAAAAAAAAAAK/2_0
ADIEVACYGYEGIDAVK/3_1(6,C,CAM)
find:  AAAAAAAAAAAAAK/2_0
ADIEVACYGYEGIDAVKEALR/2_1(6,C,CAM)
find:  AAAAAAAAAAAAAK/2_0
ADIEVACYGYEGIDAVKEALR/3_1(6,C,CAM)
find:  AAAAAAAAAAAAAK/2_0
ADIFMFDEPSSYLDVK/2_0
find:  AAAAAAAAAAAAAK/2_0
ADIFMFDEPSSYLDVK/2_1(4,M,Oxidation)
find:  AAAAAAAAAAAAAK/2_0
ADIFVDPVLHTACALDIK/2_1(12,C,CAM)
find:  AAAAAAAAAAAAAK/2_0
ADIFVDPVLHTACALDIK/3_1(12,C,CAM)
find:  AAAAAAAAAAAAAK/2_0
ADIGAPSGFK/2_0
find:  AAAAAAAAAAAAAK/2_0
ADIGCTPGSGK/2_1(4,C,CAM)
find:  AAAAAAAAAAAAAK/2_0
ADIGCTPGSGKDYAGVFSDAGLTFTSSSGQQTAQR/3_1(4,C,CAM)
find:  AAAAAAAAAAAAAK/2_0
ADIGGMLPGLTVR/2_0
find:  AAAAAAAAAAAAAK/2_0
ADIGIAMGISGSDVSK/2_0
find:  AAAAAAAAAAAAAK/2_0
ADIGIAMGISGSDVSK/2_1(6,M,Oxidation)
find:  AAAAAAAAAAAAAK/2_0
ADIGPNLDQLKDDEILR/2_0
find:  AAAAAAAAAAAAAK/2_0
ADIGPNLDQLKDDEILR/3_0
find:  AAAAAAAAAAAAAK/2_0
ADIGTPSNFQHIGHVGWDPNTGFDLNNLDPELK/3_0
find:  AAAAAAAAAAAAAK/2_0
ADIGTPSNFQHIGHVG

KeyboardInterrupt: 